In [5]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "unbalaned_68unprotected.csv"
df = pd.read_csv(filepath)
features = df

features



,page_id,page_title,view_count,protection_level,protection_expiry,page_length,page_id_scrapped,number_page_watchers,number_page_watchers_recent_edits,number_of_redirects,page_views_past_30days,total_edits,recent_number_of_edits,number_distinct_authors,number_categories
0,1749993,20th Century Fox Home Entertainment,400690,unprotected,NaN,123,65007973,64,11,0,1920,2,0,0,2
1,31140892,Made for Each Other (album),7006,unprotected,NaN,69,31140892,25,14,0,0,2,0,0,2
2,2327909,Ignorance Is Bliss (album),17107,unprotected,NaN,69,60614178,25,14,0,0,2,0,0,2
3,4600992,Andrew Clarke (British Army officer),21293,unprotected,NaN,63,63747941,25,14,0,30,2,0,0,2
4,5066067,Casa (album),16908,unprotected,NaN,54,61280680,25,14,0,0,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6909,49673183,Andrei Lavrinovich,556,unprotected,NaN,0,0,25,14,0,0,100,0,0,7
6910,49886112,"StraÃ…Â¾iÃ…Â¡Ã„Âe, Prevalje",3121,unprotected,NaN,0,0,25,14,0,0,100,0,0,7
6911,49921090,Munich Forum for Islam,1147,unprotected,NaN,0,0,25,14,0,0,100,0,0,7
6912,49952693,2015Ã¢â‚¬â€œ16 National Basketball League (Cze...,8450,unprotected,NaN,0,0,25,14,0,0,100,0,0,7


In [6]:
# Labels are the values we want to predict
labels = np.array(df['protection_level'])

# 0 => unprotected
# 1 => autoconfirmed
# 2 => extendedconfirmed
# 3 => sysop
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)  

# Remove the labels from the features
features = features.drop('protection_level', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('protection_expiry', axis = 1)
features = features.drop('page_id', axis = 1)

#np.nan convers
features = features.replace('Fewer than 30 watchers',np.NaN)
features = features.replace('There may or may not be a watching user visiting recent edits',np.NaN)

features['page_length'] = features['page_length'].astype(float)
features['total_edits'] = features['total_edits'].astype(float)
features['number_page_watchers'] = features['number_page_watchers'].astype(float)
features['number_page_watchers_recent_edits'] = features['number_page_watchers_recent_edits'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

features

array([[4.0069000e+05, 1.2300000e+02, 6.5007973e+07, 6.4000000e+01,
        1.1000000e+01, 0.0000000e+00, 1.9200000e+03, 2.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.0000000e+00],
       [7.0060000e+03, 6.9000000e+01, 3.1140892e+07, 2.5000000e+01,
        1.4000000e+01, 0.0000000e+00, 0.0000000e+00, 2.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.0000000e+00],
       [1.7107000e+04, 6.9000000e+01, 6.0614178e+07, 2.5000000e+01,
        1.4000000e+01, 0.0000000e+00, 0.0000000e+00, 2.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.0000000e+00],
       [2.1293000e+04, 6.3000000e+01, 6.3747941e+07, 2.5000000e+01,
        1.4000000e+01, 0.0000000e+00, 3.0000000e+01, 2.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.0000000e+00],
       [1.6908000e+04, 5.4000000e+01, 6.1280680e+07, 2.5000000e+01,
        1.4000000e+01, 0.0000000e+00, 0.0000000e+00, 2.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.0000000e+00],
       [3.3777000e+04, 7.0000000e+01, 6.7940642e+0

In [7]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size =0.20, random_state = 53)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels


from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:12<00:00,  2.35it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
AdaBoostClassifier                 1.00               0.58    None      1.00   
NearestCentroid                    0.99               0.58    None      0.99   
GaussianNB                         0.98               0.58    None      0.99   
BernoulliNB                        0.95               0.57    None      0.97   
XGBClassifier                      1.00               0.50    None      1.00   
LogisticRegression                 1.00               0.50    None      1.00   
LinearSVC                          1.00               0.50    None      1.00   
LGBMClassifier                     1.00               0.50    None      1.00   
DecisionTreeClassifier             1.00               0.50    None      1.00   
ExtraTreeClassifier                1.00               0.50    None      1.00   
LinearDiscriminantAnalysis         0.99 

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

Accuracy: 99.7831%
